In [1]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import cv2
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [2]:
print(cv2.__version__)

4.6.0


In [3]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops
from object_detection.models import center_net_hourglass_feature_extractor
from object_detection.utils import config_util

%matplotlib inline

In [4]:
detection_model = center_net_hourglass_feature_extractor

In [8]:
detection_model_handle = 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512/1'
model = hub.load(detection_model_handle)

OSError: SavedModel file does not exist at: C:\Users\SHOCKER\AppData\Local\Temp\tfhub_modules\3085eb2fbe2ad0b69801d50844c97b7a7a5ecade\{saved_model.pbtxt|saved_model.pb}

In [7]:
def detect_fn(image):
    """ detects object in an image"""
    image, shapes = detection_model.preproces(image)
    prediction = detection_model.predict(image)
    detections = detection_model.postprocess(prediction, shapes)
    
    return detections, prediction, tf.reshape(shapes, -1)

def detect_fn2(image):
    results = detection_model(image)
    results = {key: value.numpy() for key, value in results.items()}
    return results

In [9]:
def load_image_into_numpy_array(image):
    """loads an image and converts it to a numpy array"""
    return np.array(np.expand_dims(image, 0), dtype=np.float32)

In [10]:
path_to_labels = r'C:\Users\SHOCKER\Tensorflow tutorials\models\research\object_detection\data\mscoco_label_map.pbtxt'

category_index = label_map_util.create_category_index_from_labelmap(path_to_labels)

In [15]:
cap = cv2.VideoCapture(0)

In [14]:
while cap.isOpened():
    ret, image_np = cap.read()
    
    image_input = load_image_into_numpy_array(image_np)
    
    results = detect_fn(image_input)
    
    label_id_offset=0
    image_with_detections = image_np.copy()
    
    '''keypoint, keypoint_scores = None, None
    if 'detection_keypoints' in results:
        keypoint = results['detection_keypoints'][0]
        keypoint_scores = results['detection_keypoint_scores'][0]'''
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_with_detections,
        results['detection_boxes'][0],
        (results['detection_classes'][0] + label_id_offset).astype(int),
        results['detection_scores'][0],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.30,
        agnostic_mode=False,
        # keypoints=keypoints,
        #  keypoint_scores=keypoint_scores,
)
    
    cv2.imshow('Object Detection', cv2.resize(image_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: module 'object_detection.models.center_net_hourglass_feature_extractor' has no attribute 'preprocess'